In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [4]:
df_green = spark.read.csv('data/raw/green/*/*', header=True, inferSchema=True)

In [5]:
df_green.registerTempTable('green')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [6]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('hour', lpep_pickup_datetime) AS hour_month, 

    SUM(total_amount) AS revenue_monthly_total_amount,
    count(*) as number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [7]:
df_green_revenue.show()

+------------+-------------------+----------------------------+--------------+
|revenue_zone|         hour_month|revenue_monthly_total_amount|number_records|
+------------+-------------------+----------------------------+--------------+
|           7|2020-01-01 00:00:00|           769.7299999999997|            45|
|         217|2020-01-01 01:00:00|                       12.55|             2|
|          82|2020-01-01 03:00:00|           401.1400000000002|            34|
|         247|2020-01-01 05:00:00|          35.400000000000006|             3|
|          80|2020-01-01 09:00:00|                      554.73|            15|
|          72|2020-01-01 09:00:00|                         6.3|             1|
|         167|2020-01-01 11:00:00|                         0.0|             1|
|         131|2020-01-01 18:00:00|                        10.3|             1|
|           7|2020-01-01 19:00:00|                      182.14|            17|
|          95|2020-01-02 13:00:00|          372.0500

In [8]:
df_green_revenue.write.parquet('data/report/revenue/green', mode = 'overwrite')

In [9]:
df_yellow = spark.read.csv('data/raw/yellow/*/*', header=True, inferSchema=True)
df_yellow.registerTempTable('yellow')

C:\tools\spark-3.3.2-bin-hadoop3\python\pyspark\sql\dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [10]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('hour', tpep_pickup_datetime) AS hour_month, 

    SUM(total_amount) AS revenue_monthly_total_amount,
    count(*) as number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [11]:
df_yellow_revenue.show()

+------------+-------------------+----------------------------+--------------+
|revenue_zone|         hour_month|revenue_monthly_total_amount|number_records|
+------------+-------------------+----------------------------+--------------+
|           7|2020-01-01 00:00:00|           455.1700000000002|            38|
|         249|2020-01-01 01:00:00|           5515.590000000013|           272|
|         217|2020-01-01 01:00:00|          41.400000000000006|             3|
|          82|2020-01-01 03:00:00|          211.70000000000005|            14|
|         247|2020-01-01 05:00:00|                         8.8|             1|
|          79|2020-01-01 07:00:00|          2021.6899999999978|           106|
|         163|2020-01-01 07:00:00|          1355.3999999999999|            45|
|         217|2020-01-01 07:00:00|                        22.3|             1|
|          80|2020-01-01 09:00:00|                       81.92|             4|
|          24|2020-01-01 10:00:00|                  

In [12]:
df_yellow_revenue.write.parquet('data/report/revenue/yellow', mode = 'overwrite')